In [1]:
using Pkg
Pkg.activate(".")

  Activating environment at `c:\Users\gabri\Documents\Cadeiras\MAT55\MAT55_Exame\Project.toml`


In [2]:
Pkg.precompile()

In [6]:
using DataFrames
using LinearAlgebra
using Gadfly
using Statistics
using MAT55_Exame

In [7]:
df = create_returns()

,date,MMM,ABT,ABBV,ABMD,ACN
,Date…,Float64,Float64,Float64,Float64,Float64
1,2021-09-21,-0.0116878,0.00208212,0.00704887,-0.00688012,-0.00343078
2,2021-09-22,0.00795874,-0.00383597,-0.00690619,0.0115076,0.0105995
3,2021-09-23,0.00939726,0.00521461,0.0089277,0.00472832,0.0249208
4,2021-09-24,-0.00269931,-0.00742219,-0.0027012,-0.000713043,-0.0109912
5,2021-09-27,0.00414273,-0.0304736,0.0060708,-0.0339365,-0.0139139
6,2021-09-28,-0.00957143,-0.0152596,-0.00352771,-0.0212722,-0.0307914
7,2021-09-29,0.00460984,0.00606368,0.0139743,-0.00531282,-0.00209738
8,2021-09-30,-0.0301858,-0.0111335,-0.0089121,-0.012109,-0.0111581
9,2021-10-01,0.00718273,-0.00778801,0.0113098,-0.0067277,0.0154413


In [16]:
# checar se há alguma correlação perfeita entre duas ações
correl = cor(Matrix(select(df, Not(:date))))
nodiag = correl - diagm(diag(correl))
findall(x -> x == 1.0, nodiag)

CartesianIndex{2}[]

In [10]:
μ, Σ, tickers, dates = get_statistics(df)

,date
,Date…
1,2021-09-21
2,2021-09-22
3,2021-09-23
4,2021-09-24
5,2021-09-27
6,2021-09-28
7,2021-09-29
8,2021-09-30
9,2021-10-01


In [ ]:
function vetor_port(σ::Array{Float64,2})
    n = size[σ,1] 
    vec1 = ones(n)
    vec2 = [ones(n); 0.0]
    b = [zero(n); 1.0]

    top_A = hcat(2*σ,vec1)
    A = vcat(top_A,vec2)

    U, S, V = svd(A)

    S_1 = Diagonal(S)

    x = V*inv(S_1)*U'*b #Vetor dos portfólios e o λ (n lembro o nome kk)

    m = x[1:n-1] # Vetor do peso dos portfólios para a menor variância global

end

In [ ]:
function gmin(μ::Array{Float64,1},m::Array{Float64,1})

    # Valor esperado para o vetor m
    μ_gmin = dot(m,μ)

    # Desvio padrão do vetor M
    σ_gmin2 = m'*σ*m
    σ_gmin = sqrt(σ_gmin2)

end

In [ ]:
function grafico1(μ::Array{Float64,1}, σ::Array{Float64,2})

    σ_vec = sqrt.(diag(σ))

    xvec = ones(3)/3      # Vetor de pesos X
    μ_x = dot(xvec, μ)
    σ_x2= xvec'*σ*xvec

    σ_x = sqrt(σ_x2)       # Desvio padrão dos pesos X
    
    σ_n = [σ_vec;σ_x]
    μ_n = [μ;μ_x]
    
    nome = ["Ação 1"; "Ação 2"; "Ação 3"; "Portfólio"]
    
    Gadfly.plot(x=σ_n, y=μ_n, label=nome, Coord.cartesian(xmin=0, xmax=0.2, ymin=0, ymax=0.05), 
        Guide.xlabel("σ"), Guide.ylabel("μ"), Guide.title("Ações e Portfólio"), Geom.point, Geom.label)

end

In [ ]:
a = [0.0427; 0.0015; 0.0285]
b = [0.0100 0.0018 0.0011; 0.0018 0.0109 0.0026; 0.0011 0.0026 0.0199]

grafico1(a, b)